### Block Metropolis Implementation

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import graphviz
import lingam
from lingam.utils import make_dot
import scipy.stats as st 
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(suppress=True)

def mh_step(last, logjoint, proposer, logproposal):
    ## chain is a list of current markov chain
    ## logjoint evaluates the log of the joint probability, log p(x)
    ## proposal generates a proposal x^* given the last x. 

    next = proposer(last) 

    logratio = logjoint(next) - logjoint(last) + logproposal(last, next) - logproposal(next, last)

    logratio = np.maximum(np.minimum(logratio, 0), -999999)

    a = np.random.rand()

    if logratio > np.log(a):
        return next, logratio
    else:
        return last, logratio

def block_mh_step(chain, logjoint_list, proposer_list, logproposal_list, exact_list):
    ## chain is a list of lists of markov chains representing each block
    ## logjoint, proposer, logproposal all lists of # of blocks
    ## If block is 'exact', accept the proposal with probability 1
    curr = chain[-1].copy()
    logratios = []
    for i in range(len(curr)):
        last = curr[i]
        cond = curr[:i] + curr[(i+1):]
        if(exact_list[i]):
            curr[i], logratio = proposer_list[i](last, cond), 0
        else:
            logjoint, proposer, logproposal = (lambda x: logjoint_list[i](x, cond)), (lambda x: proposer_list[i](x, cond)), (lambda x, y: logproposal_list[i](x, y, cond))
            curr[i], logratio = mh_step(last, logjoint, proposer, logproposal)
        logratios.append(logratio)        
    chain.append(curr)
    return logratios


### Generating Synthetic Data

In [ ]:
### synthetic data generation

np.random.seed(7)

N = 5 ## dimension
d = np.sum(np.arange(N)) ## possible nonzero entries
p = np.repeat(0.2, d) # sparsity
n = 50 ## samples

## fixed P, L

P = [0, 4, 3, 2, 1]
L = [2, 0, 3, 0, 1, 2, 0, 0, -1, -3]

def range_2_mat(entries):
    P = np.identity(N)
    P = P[entries]
    return P

def vec_2_LT(entries):
    L = np.zeros((N, N))
    L[np.tril_indices(N, -1)] = entries
    return L

def LT_2_vec(L):
    return L[np.tril_indices(N, -1)]

def A_from_PL(P, L):
    P_mat = range_2_mat(P)
    L_mat = vec_2_LT(L)
    return P_mat @ L_mat @ P_mat.T


def gen_data_fixedPL(P, L, n):
    A = A_from_PL(P, L)
    mat = np.identity(N) - A
    eps = np.random.randn(N,n)
    X_t = np.linalg.inv(mat) @ eps

    return np.transpose(X_t), P, L

X, P, L = gen_data_fixedPL(P, L, n)

### loglikelihood for data

def loglike(X, P, L):
    P_mat = range_2_mat(P)
    L_mat = vec_2_LT(L)
    prec = np.identity(N) + P_mat @ (L_mat.T @ L_mat - (L_mat + L_mat.T)) @ P_mat.T
    cov = np.linalg.inv(prec)
    loglike = st.multivariate_normal.logpdf(X, cov = cov)
    return loglike


### Defining Targets and Proposals

In [ ]:

### global prior parameters

p_prior = np.repeat(0.2, d)
sigma_prior = 1
spike_ratio = 10


## uncomment for identifiable model

#for i in (i+sum(range(i+1)) for i in (range (N-1))):
#    p_prior[i] = 1


### helpers for locally balanced proposer

def list_of_swaps(N):
    swaps = []
    for i in range(N):
        for j in range(N):
            if i < j:
                swaps.append((i,j))
    return swaps

def swap_permutation(P, swap):
    P_swapped = P.copy()
    P_swapped[swap[0]], P_swapped[swap[1]] = P_swapped[swap[1]], P_swapped[swap[0]]

    return P_swapped

def LB_weights(P_last, pi, swaps):
    dim = int(N*(N-1)/2)
    logprobs = np.zeros(dim)
    for i, swap in enumerate(swaps):
        P = swap_permutation(P_last, swap)
        logprobs[i] = pi(P) 
    sumlogprob = sp.special.logsumexp(logprobs)
    logprobs += -sumlogprob

    return logprobs, sumlogprob

def proposer_perm(last, L, swaps):
    pi = lambda x: loglike(X, x, L).mean()
    weights = LB_weights(last, pi, swaps)
    sampling_weights = np.exp(weights[0])
    sampleid = np.random.choice(len(swaps), p = sampling_weights)
    return swap_permutation(last, swaps[sampleid]), weights[1]

swaps = list_of_swaps(N)

### building blocks for all models.
### parametrization is (I, P, L). 
### each function takes in (last = I, cond = [other args]). 

### I: indicator variables

logjoint_I = None

def proposer_I(last, cond):
    L = cond[1].copy()
    next = np.zeros_like(last)
    for id, l in enumerate(L):
        p = p_prior[id]*(st.norm.pdf(l, loc = 0, scale = sigma_prior)/(st.norm.pdf(l, loc = 0, scale = sigma_prior) + st.norm.pdf(l, loc = 0, scale = sigma_prior/spike_ratio)))
        next[id] = np.random.binomial(1, p)
    return next

logproposal_I = None 

### P: permutation locally balanced

def logjoint_P(x, cond):
    L = cond[1].copy()
    return loglike(X, x, L).mean()

def proposer_P(last, cond):
    L = cond[1].copy()
    return proposer_perm(last, L, swaps)[0]

def logproposal_P(next, last, cond):
    L = cond[1].copy()
    return ((1/2)*(logjoint_P(next, cond) - logjoint_P(last, cond)) - proposer_perm(last, L, swaps)[1])

### L: weights

def logjoint_L(x, cond):
    I = cond[0].copy()
    P = cond[1].copy()
    loglikes = loglike(X, P, x).mean()
    for id, i in enumerate(I):
        if i == 0:
            loglikes += np.maximum(np.log(st.norm.pdf(x[id], loc = 0, scale = sigma_prior/spike_ratio)), -9999999)/n
        else:
            loglikes += np.maximum(np.log(st.norm.pdf(x[id], loc = 0, scale = sigma_prior)), -9999999)/n
    return loglikes

def proposer_L(last, cond):
    I = cond[0].copy()
    next = np.empty_like(last)
    for id, i in enumerate(I):
        if i == 0:
            next[id] = last[id] + np.random.randn()/spike_ratio
        else:
            next[id] = last[id] + np.random.randn()
    return next

def logproposal_L(next, last, cond):
    I = cond[0].copy()
    logprob = 0
    for id, i in enumerate(I):
        if i == 0:
            logprob += np.log(st.norm.pdf(next[id], loc = last[id], scale = 1/spike_ratio))
        else:
            logprob += np.log(st.norm.pdf(next[id], loc = last[id], scale = 1))  
    return logprob


logjoint_list = [logjoint_I, logjoint_P, logjoint_L]

proposer_list = [proposer_I, proposer_P, proposer_L]

logproposal_list = [logproposal_I, logproposal_P, logproposal_L]

exact_list = [True, False, False]

### Running MCMC

In [ ]:
chain = [ [np.zeros(d).astype(int), np.random.permutation(N), np.random.randn(d)] ]

logratios = []

iters = 5000

for i in range(iters):
    logratios.append(block_mh_step(chain, logjoint_list, proposer_list, logproposal_list, exact_list))
    if (i % 10 == 0):
        print(f' Iterate {i} of {iters}', end = "\r")


### Post processing

In [ ]:
### post processing

burn_in = int(iters/2)

chainburn = chain[burn_in:]

chain_I, chain_P, chain_L = np.array([iter[0] for iter in chainburn]), np.array([iter[1] for iter in chainburn]), np.array([iter[2] for iter in chainburn])

### convert to pandas df
I_df, P_df, L_df = pd.DataFrame(chain_I), pd.DataFrame(chain_P), pd.DataFrame(chain_L)

P_mode = P_df.mode(axis = 0).values.tolist()[0]
L_mean = L_df.mean(axis = 0).values.tolist()

A_bayes = A_from_PL(P_mode, L_mean)

In [ ]:
## distribution of P
P_df.value_counts()

In [ ]:
P_mode

In [ ]:
## acceptance rate
np.exp(np.array(logratios)).mean(axis = 0)

### Drawing Graphs


In [ ]:
model = lingam.DirectLiNGAM()
model.fit(X)
graph_lingam = make_dot(model.adjacency_matrix_, lower_limit = 0.1)
graph_lingam.attr(rankdir='LR')
graph_lingam

In [ ]:
true_A = A_from_PL(P, L)
graph_true = make_dot(true_A, lower_limit = 0.1)
graph_true.attr(rankdir='LR')
graph_true

In [ ]:
graph_post = make_dot(A_bayes, lower_limit = 0.1)
graph_post.attr(rankdir = 'LR')
graph_post